In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
import re
import pandas as pd
import numpy as np

# Set Chrome options and the user-agent
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

# Initialize the ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)


origin = "SCL"
destination = "MXP"
startdate = "2023-11-07"
enddate = "2023-11-14"
url = f"https://www.kayak.cl/flights/{origin}-{destination}/{startdate}/{enddate}?sort=bestflight_a"


# Open the Kayak website
driver.get(url)
# sleep(13)

while True:
    try:
        boton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@role="button" and @class="ULvh-button show-more-button"]')))
        boton.click()
    except:
        # Handle the situation when the button disappears or is no longer clickable
        print("Button disappeared or not clickable. Stopping.")
        break


# sleep(5) 
# soup=BeautifulSoup(driver.page_source, 'html.parser')

# if soup.find_all('p')[0].getText() == "Please confirm that you are a real KAYAK user.":
#     print("Kayak thinks I'm a bot, which I am ... so let's wait a bit and try again")
#     driver.close()
#     sleep(20)
#     return "failure"




soup=BeautifulSoup(driver.page_source, 'html.parser')
deptime = []
arrtime = []

# Find all div elements with class "vmXl vmXl-mod-variant-large"
div_elements_times = soup.find_all('div', class_='vmXl vmXl-mod-variant-large')

# Iterate through the div elements
for div in div_elements_times:
    # Find the first and second spans within each div
    spans = div.find_all('span')
    
    if len(spans) == 3:
        # Extract the text from the first span and append it to deptime
        deptime.append(spans[0].get_text())
        
        # Extract the text from the second span and append it to arrtime
        arrtime.append(spans[2].get_text())




div_elements_price = soup.find_all('div', class_='f8F1-price-text')

prices_int = []

for div in div_elements_price:
    # Extract the text from the div
    price_text = div.get_text()
    
    # Remove any non-numeric characters (e.g., dollar sign and comma)
    price_text = price_text.replace('$', '').replace('.', '')
      
    # Append the integer to the list
    prices_int.append(int(price_text))


Button disappeared or not clickable. Stopping.


In [35]:
print(len(deptime))
print(len(arrtime))
print(len(prices_int))

228
228
114
